## Simple Mission in Auto-Mode

This notebook serves to check the ardupilot installation

In [ ]:
import os

from config import (
    ARDU_LOGS_PATH,
    ARDUPILOT_VEHICLE_PATH,
    ENV_CMD_PYT,
    VEH_PARAMS_PATH,
    BasePort,
)
from helpers import clean, create_process, setup_logging, wait_for_port
from mavlink.connections import create_tcp_conn
from plan import State
from plan.planner import Plan

clean()

## Launch Copter (ardupilot)

In [2]:
#This must agree with first waypoint in mission.waypoint
spawn_str= '-35.3633245,149.1652241,0.0,0' 

sim_vehicle_path = os.path.expanduser(ARDUPILOT_VEHICLE_PATH)
sysid = 1
vehicle_cmd = (
    f"python3 {sim_vehicle_path} "
    f"-v ArduCopter "
    f"-I{sysid-1} --sysid {sysid} "
    f"--no-rebuild "
    f"--use-dir={ARDU_LOGS_PATH} "
    f"--add-param-file {VEH_PARAMS_PATH} "
    f"--no-mavproxy "
    f"--port-offset=0 "  # ArduPilot automatically binds TCP port 5760 (BasePort.ARP)
    f'--custom-location={spawn_str}'
)

create_process(
                vehicle_cmd,
                after="exec bash",
                visible= True,
                suppress_output=False,
                title="ardu_vehicle",
                env_cmd=ENV_CMD_PYT,
            )
wait_for_port(port = BasePort.ARP, timeout = 0.5)

Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting fo

## 1. Connect to the vehicle

In [ ]:
conn = create_tcp_conn(
    base_port=BasePort.ARP, 
    offset=0,
    role="client", 
    sysid=sysid
)

print("✅ TCP connection established!")

✅ TCP connection established!


## Create auto plan from mission file

The mission is loaded from "plan/missions/simple_mission.waypoints"

In [4]:
plan = Plan.auto(name='auto',
                mission_path="plan/missions/simple_mission.waypoints",
                monitored_items=[1,2])
plan

🕓 <Plan '📋 auto'>
  🕓 <Action '📤 UPLOAD_MISSION'>
    🕓 <Step '🔹 clear uav missions'>
    🕓 <Step '🔹 load mission plan/missions/simple_mission.waypoints'>
  🕓 <Action '🔧 PREARM'>
    🕓 <Step '🔹 Check disarmed'>
    🕓 <Step '🔹 Check EKF status'>
    🕓 <Step '🔹 Check GPS'>
    🕓 <Step '🔹 Check system'>
  🕓 <Action '⚙️  MODE: GUIDED'>
    🕓 <Step '🔹 Switch to GUIDED'>
  🕓 <Action '🔐 ARM'>
    🕓 <Step '🔹 arm'>
  🕓 <Action '🚀 START_MISSION'>
    🕓 <Step '🔹 start_mission'>
  🕓 <Action '👁️ MONITOR_MISSION'>
    🕓 <Step '🔹 check item 1'>
    🕓 <Step '🔹 check item 2'>
    🕓 <Step '🔹 check mission completion'>

# Execute Auto Plan

In [5]:
plan.bind(conn)
setup_logging('plan',verbose=1)
while plan.state != State.DONE:
    plan.act()

15:24:17 - plan - INFO - 🧹 Vehicle 1: Cleared previous mission
15:24:17 - plan - INFO - ✅ Vehicle 1: 4 waypoints read
15:24:18 - plan - INFO - ✅ Vehicle 1: Mission upload successful!
15:24:18 - plan - INFO - ✅ Vehicle 1: Action Done: 📤 UPLOAD_MISSION
15:24:59 - plan - INFO - ✅ Vehicle 1: Action Done: 🔧 PREARM
15:24:59 - plan - INFO - ✅ Vehicle 1: Action Done: ⚙️  MODE: GUIDED
15:25:00 - plan - INFO - ✅ Vehicle 1: Action Done: 🔐 ARM
15:25:00 - plan - INFO - 🚀 Vehicle 1: Mission has started
15:25:00 - plan - INFO - ✅ Vehicle 1: Action Done: 🚀 START_MISSION
15:25:07 - plan - INFO - Vehicle 1: ⭐ Reached waypoint: 1
15:25:14 - plan - INFO - Vehicle 1: ⭐ Reached waypoint: 2
15:25:26 - plan - INFO - Vehicle 1: Mission completed
15:25:26 - plan - INFO - ✅ Vehicle 1: Action Done: 👁️ MONITOR_MISSION
15:25:26 - plan - INFO - ✅ Vehicle 1: Plan Done: 📋 auto


In [6]:
plan

✅ <Plan '📋 auto'>
  ✅ <Action '📤 UPLOAD_MISSION'>
    ✅ <Step '🔹 clear uav missions'>
    ✅ <Step '🔹 load mission plan/missions/simple_mission.waypoints'>
  ✅ <Action '🔧 PREARM'>
    ✅ <Step '🔹 Check disarmed'>
    ✅ <Step '🔹 Check EKF status'>
    ✅ <Step '🔹 Check GPS'>
    ✅ <Step '🔹 Check system'>
  ✅ <Action '⚙️  MODE: GUIDED'>
    ✅ <Step '🔹 Switch to GUIDED'>
  ✅ <Action '🔐 ARM'>
    ✅ <Step '🔹 arm'>
  ✅ <Action '🚀 START_MISSION'>
    ✅ <Step '🔹 start_mission'>
  ✅ <Action '👁️ MONITOR_MISSION'>
    ✅ <Step '🔹 check item 1'>
    ✅ <Step '🔹 check item 2'>
    ✅ <Step '🔹 check mission completion'>